In [1]:
#OpenCV module
import cv2
#Modulo para leer directorios y rutas de archivos
import os
#OpenCV trabaja con arreglos de numpy
import numpy
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

captura

In [10]:
#OpenCV module
import cv2
#Modulo para leer directorios y rutas de archivos
import os
#OpenCV trabaja con arreglos de numpy
import numpy
#Obtener el nombre de la persona que estamos capturando
import sys
nombre = 'antonio'

#Directorio donde se encuentra la carpeta con el nombre de la persona
dir_faces = './caras/'
path = os.path.join(dir_faces, nombre)

#Tamaño para reducir a miniaturas las fotografias
size = 4

#Si no hay una carpeta con el nombre ingresado entonces se crea
if not os.path.isdir(path):
    os.mkdir(path)

#cargamos la plantilla e inicializamos la webcam
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

img_width, img_height = 112, 92

#Ciclo para tomar fotografias
count = 0
while count < 100:
    #leemos un frame y lo guardamos
    rval, img = cap.read()
    img = cv2.flip(img, 1, 0)

    #convertimos la imagen a blanco y negro
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #redimensionar la imagen
    mini = cv2.resize(gray, (int(gray.shape[1] / size), int(gray.shape[0] / size)))

    """buscamos las coordenadas de los rostros (si los hay) y
   guardamos su posicion"""
    faces = face_cascade.detectMultiScale(mini)    
    faces = sorted(faces, key=lambda x: x[3])
    
    if faces:
        face_i = faces[0]
        (x, y, w, h) = [v * size for v in face_i]
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (img_width, img_height))
        
        #Dibujamos un rectangulo en las coordenadas del rostro
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
        #Ponemos el nombre en el rectagulo
        cv2.putText(img, nombre, (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN,1,(0, 255, 0))        

        #El nombre de cada foto es el numero del ciclo
        #Obtenemos el nombre de la foto
        #Despues de la ultima sumamos 1 para continuar con los demas nombres
        # Filtrar solo nombres numéricos y convertirlos a enteros
        numeros = [int(n.split('.')[0]) for n in os.listdir(path) if n.split('.')[0].isdigit()]

        # Si la lista está vacía, empezar desde 1; de lo contrario, obtener el siguiente número
        pin = max(numeros, default=0) + 1

        #Metemos la foto en el directorio
        print
        cv2.imwrite('%s/%s.png' % (path, pin), face_resize)

        #Contador del ciclo
        count += 1

    #Mostramos la imagen
    cv2.imshow('OpenCV Entrenamiento de '+nombre, img)

    #Si se presiona la tecla ESC se cierra el programa
    key = cv2.waitKey(10)
    if key == 27:
        cv2.destroyAllWindows()
        break

video

In [5]:
import cv2
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Parte 1: Creando el entrenamiento del modelo
print('Formando...')

# Directorio donde se encuentran las carpetas con las caras de entrenamiento
dir_faces = './caras/'

# Tamaño para reducir a miniaturas las fotografías
size = 4

# Crear una lista de imágenes y una lista de nombres correspondientes
(images, labels, names, id) = ([], [], {}, 0)
for (subdirs, dirs, files) in os.walk(dir_faces):
    for subdir in dirs:
        names[id] = subdir
        subjectpath = os.path.join(dir_faces, subdir)
        for filename in os.listdir(subjectpath):
            path = os.path.join(subjectpath, filename)
            label = id
            img = cv2.imread(path, 0)
            img = cv2.resize(img, (112, 92))
            images.append(img.flatten())  # Convertimos la imagen en un vector
            labels.append(int(label))
        id += 1

(im_width, im_height) = (112, 92)

# Convertir a arrays de numpy
images = np.array(images)
labels = np.array(labels)

def compare_faces(face_vector, images, labels, names):
    similarities = cosine_similarity([face_vector], images)
    best_match_idx = np.argmax(similarities)
    best_match_score = similarities[0][best_match_idx]
    
    if best_match_score > 0.99:  # Umbral de similitud
        return names[labels[best_match_idx]], best_match_score
    else:
        return "Desconocido", best_match_score

# Parte 2: Utilizar el modelo entrenado en funcionamiento con la cámara
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    # Leemos un frame y lo guardamos
    rval, frame = cap.read()
    frame = cv2.flip(frame, 1, 0)

    # Convertimos la imagen a blanco y negro    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Redimensionar la imagen
    mini = cv2.resize(gray, (int(gray.shape[1] / size), int(gray.shape[0] / size)))

    # Buscamos las coordenadas de los rostros (si los hay) y guardamos su posición
    faces = face_cascade.detectMultiScale(mini)
    
    for i in range(len(faces)):
        face_i = faces[i]
        (x, y, w, h) = [v * size for v in face_i]
        face = gray[y:y + h, x:x + w]
        face_resize = cv2.resize(face, (im_width, im_height))
        face_vector = face_resize.flatten()  # Convertir la imagen en un vector

        # Comparar con las caras de entrenamiento
        cara, score = compare_faces(face_vector, images, labels, names)
        
        # Dibujar rectángulo y mostrar nombre
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
        cv2.putText(frame, f'{cara} - {score:.2f}', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
        
    cv2.imshow('OpenCV Reconocimiento facial', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Formando...


In [ ]:
import os
import numpy as np
from typing import Dict, List, Any
from mlserver import MLModel
from mlserver.types import InferenceRequest, InferenceResponse, ResponseOutput
from mlserver.errors import InferenceError
import json

import cv2
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Import the detector class
from yolo_detector import YOLOObjectDetector


class ObjectDetectionModel(MLModel):
    
    async def load(self) -> bool:
        # Create the detector instance directly
        self.model = YOLOObjectDetector()
        self.model2 = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

        # No need to load it now - will load on first prediction
        self.ready = True
        return self.ready

    async def predict_caras(self, frame, boundingBox) -> bool:
        
        # Get predictions
        x, y, w, h = boundingBox
        mini = frame[y:y + h, x:x + w]
        predictions = self.model2.detectMultiScale(mini)

        face_i = predictions[0]
        (x2, y2, w2, h2) = [v * size for v in face_i]
        face = frame[y2:y2 + h2, x2:x2 + w2]
        face_resize = cv2.resize(face, (112, 92))
        face_vector = face_resize.flatten()  # Convertir la imagen en un vector
        
        return face_vector

    async def predict(self, payload: InferenceRequest) -> InferenceResponse:
        try:
            # Extract images from payload
            inputs = payload.inputs[0]
            image_data = inputs.data

            # Get predictions
            predictions = self.model2.detectMultiScale(mini)

            face_i = predictions[0]
            (x, y, w, h) = [v * size for v in face_i]
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (im_width, im_height))
            face_vector = face_resize.flatten()  # Convertir la imagen en un vector
            
            # Format response
            return InferenceResponse(
                model_name=self.name,
                model_version=self._settings.parameters.version,
                outputs=[
                    ResponseOutput(
                        name="detections",
                        shape=[len(predictions)],
                        datatype="BYTES",
                        data=[json.dumps(pred) for pred in predictions],
                    )
                ],
            )
        except Exception as e:
            raise InferenceError(f"Error during inference: {str(e)}")